In [1]:
import pandas as pd
import os
import re
import json
from typing import List, Optional, Tuple

In [2]:
def normalize_trait(trait):
    trait = '_'.join(trait.split())
    normalized_trait = ''.join(trait.split("'"))
    return normalized_trait

## Get trait-condition pairs and get gene IoUs

In [3]:
with open("../metadata/task_info.json", "r") as f:
    info = json.load(f)
t2g = {k: v['related_genes'] for k, v in info.items()}
traits = sorted(list(t2g.keys()))

In [ ]:
len(traits)

In [ ]:
len(t2g['Breast_Cancer'])

In [ ]:
cancer_traits = \
['Acute_Myeloid_Leukemia', 'Adrenocortical_Cancer', 'Bile_Duct_Cancer', 'Bladder_Cancer', 'Breast_Cancer', 
'Cervical_Cancer', 'Colon_and_Rectal_Cancer', 'Endometrioid_Cancer', 'Esophageal_Cancer', 'Glioblastoma', 
'Head_and_Neck_Cancer', 'Kidney_Chromophobe', 'Kidney_Clear_Cell_Carcinoma', 'Kidney_Papillary_Cell_Carcinoma', 
'Large_B-cell_Lymphoma', 'Liver_Cancer', 'Lower_Grade_Glioma', 'Lung_Cancer', 'Melanoma', 'Mesothelioma', 
'Ocular_Melanomas', 'Ovarian_Cancer', 'Pancreatic_Cancer', 'Pheochromocytoma_and_Paraganglioma', 
'Prostate_Cancer', 'Rectal_Cancer', 'Retinoblastoma', 'Sarcoma', 'Stomach_Cancer', 'Testicular_Cancer', 
'Thymoma', 'Thyroid_Cancer', 'Uterine_Carcinosarcoma', 'Uterine_Corpus_Endometrial_Carcinoma', 
'lower_grade_glioma_and_glioblastoma']
len(cancer_traits)

In [7]:
data = []
for t in traits:
    for s in traits + ['Age', 'Gender']:
        if t == s: continue
        if s in ['Age', 'Gender']: 
            iou = 1.0
        else:
            len_union = len(set(t2g[t]).union(set(t2g[s])))
            if len_union == 0: continue
            iou = len(set(t2g[t]).intersection(set(t2g[s]))) / len_union
        data.append({'Trait': t, 'Condition': s, 'IoU': iou})
rel = pd.DataFrame(data)

In [ ]:
rel

In [9]:
# 回头修订一下。

condition_only = ['Vitamin_D_Levels', 'LDL_Cholesterol_Levels']
male_traits = ['Prostate_Cancer', 'Testicular_Cancer']
female_traits = ['Cervical_Cancer', 'Endometriosis', 'Endometrioid_Cancer', 'Uterine_Carcinosarcoma', 'Uterine_Corpus_Endometrial_Carcinoma', 'Ovarian_Cancer', 'Polycystic_Ovary_Syndrome'] 
gender_traits = male_traits + female_traits
condition_all = ['Obesity', 'Hypertension']

In [10]:
assert all(t in traits for t in condition_only)
assert all(t in traits for t in gender_traits)
assert all(t in traits for t in condition_all)

In [ ]:
rel = rel[~((rel['Trait'].isin(gender_traits)) & (rel['Condition'] == 'Gender'))]
rel = rel[~rel['Trait'].isin(condition_only)]
rel

In [ ]:
rel = rel[~(rel['Trait'].isin(male_traits) & rel['Condition'].isin(female_traits))]
rel = rel[~(rel['Trait'].isin(female_traits) & rel['Condition'].isin(male_traits))]
rel

In [ ]:
rel = rel[~((rel['Trait'].isin(cancer_traits)) & (rel['Condition'].isin(cancer_traits)))]
rel

In [ ]:
rel = rel.sort_values(by='IoU', ascending=False).reset_index().drop(columns=['index'])
rel

In [ ]:
selected = rel[(rel['IoU'] >= 0.20) | rel['Condition'].isin(condition_all)]
selected

In [17]:
# Create a new dictionary to store both genes and conditions for each trait
new_task_info = {}

# Iterate through each trait in t2g
for trait in t2g:
    # Get the conditions for this trait from the selected DataFrame
    trait_conditions = selected.loc[selected['Trait'] == trait, 'Condition'].unique().tolist()
    
    # Create the new dictionary structure for this trait
    new_task_info[trait] = {
        'related_genes': t2g[trait],  # Original list of genes
        'conditions': trait_conditions       # List of conditions
    }

In [ ]:
len(new_task_info['Acute_Myeloid_Leukemia']['related_genes'])

In [ ]:
new_task_info['Height']['related_genes']

In [ ]:
for k in new_task_info:
    print(k, len(new_task_info[k]['related_genes']))

In [18]:
with open("../metadata/task_info.json", "w") as f:
    json.dump(new_task_info, f, indent=4)

In [19]:
with open("../metadata/task_info.json", "r") as f:
    new_task_info2 = json.load(f)

In [ ]:
new_task_info == new_task_info2

### Refine trait-condition pairs
Remove those that don't have common gene regressors.

In [1]:
import sys
sys.path.append('..')
from tools.statistics import *
from utils.utils import get_question_pairs

ModuleNotFoundError: No module named 'seaborn'

In [4]:
data_root = "../output/preprocess/1"
task_info_path = "../metadata/task_info.json"
with open(task_info_path, "r") as f:
    task_info = json.load(f)

In [7]:
'Hypertension' in task_info['Hypertension']['conditions']

False

In [ ]:
data_root = "../output/preprocess/1"
task_info_path = "../metadata/task_info.json"
with open(task_info_path, "r") as f:
    task_info = json.load(f)

count = 0
all_traits = sorted(list(task_info.keys()))
for trait in all_traits:
    for condition in task_info[trait]['conditions']:
        if condition in ['Age', 'Gender', None]: continue
        try:
            trait_data, condition_data, regressor = select_and_load_cohort(data_root, trait, condition, True, task_info_path)
            if regressor is None:
                print(f"NO REGRESSORS FOR {trait} and {condition}" )
                count += 1
                print(count)
        except:
            continue
print(count)


Found 331 candidate genes that can be used in two-step regression analysis, such as ['AGTR1', 'ADRB1', 'ACE', 'EDNRA', 'ADRA1A', 'GUCY1B1', 'ADRA1B', 'NR3C2', 'REN', 'CACNA1C'].
Found 45 candidate genes that can be used in two-step regression analysis, such as ['THPO', 'MPL', 'JAK2', 'PDE3A', 'PDE3B', 'CALR', 'RRM2', 'SH2B3', 'TET2', 'JAK1'].
Found 330 candidate genes that can be used in two-step regression analysis, such as ['AGTR1', 'ADRB1', 'ACE', 'EDNRA', 'ADRA1A', 'GUCY1B1', 'ADRA1B', 'NR3C2', 'REN', 'CACNA1C'].
Found 329 candidate genes that can be used in two-step regression analysis, such as ['AGTR1', 'ADRB1', 'ACE', 'EDNRA', 'ADRA1A', 'GUCY1B1', 'ADRA1B', 'NR3C2', 'REN', 'CACNA1C'].
No available data, best cohorts being 'None' for the trait 'Alcohol_Flush_Reaction' and 'GSE128381' for the condition 'Hypertension'
NO REGRESSORS FOR Alcohol_Flush_Reaction and Hypertension
1
Found 16 candidate genes that can be used in two-step regression analysis, such as ['ACE', 'GUCY1B1', 'GUC

In [ ]:
with open(file_path, 'r') as f:
    task_info = json.load(f)
        